In [22]:
import re

import pandas as pd

In [23]:
def read_tbl():
    df = pd.read_excel(
        "../../../../data/NM/23-6-26/Copy_of_IPRA_Request_PO16267_FullData__Sheets.xlsx"
    )
    return df


df = read_tbl()

# Select the columns needed for processing
person_columns = [
    "Person Certification #",
    "Person First Name",
    "Person Middle Name",
    "Person Last Name",
    "Person Suffix",
    "Person Gender",
    "Person Date of Birth",
    "Year of Birth",
    "Person EEOC Category",
]

# Define the base names for employment columns
employment_base_columns = [
    "Employment Start Date",
    "Employment End Date",
    "Employment Appointment Type*",
    "Employment Employment Type*",
    "Employment Title/Rank (Current)",
    "Employment Status",
    "Is Primary Employment",
    "Employing Organization Name",
    "Employing Organization Agency Type",
]

# Function to get all employment-related columns


def get_employment_columns(df):
    employment_columns = []
    for col in df.columns:
        if (
            col.startswith("Employment")
            or col.startswith("Employing")
            or col == "Is Primary Employment"
        ):
            employment_columns.append(col)
    return employment_columns


# Get all employment-related columns
employment_columns = get_employment_columns(df)

# Combine person columns and employment columns
columns_to_keep = person_columns + employment_columns

# Filter the dataframe to keep only the relevant columns
filtered_df = df[columns_to_keep]

# Create an empty DataFrame to store the cleaned data
cleaned_df = pd.DataFrame()

# Function to get the base column name


def get_base_column(col):
    for base in employment_base_columns:
        if col.startswith(base):
            return base
    return col


# Function to extract number from column name


def extract_number(col):
    match = re.search(r"\d+$", col)
    return int(match.group()) if match else 0


# Group employment columns
employment_groups = {}
for col in employment_columns:
    base_col = get_base_column(col)
    if base_col not in employment_groups:
        employment_groups[base_col] = []
    employment_groups[base_col].append(col)

# Sort columns in each group
for base_col in employment_groups:
    employment_groups[base_col].sort(key=extract_number)

# Determine the maximum number of employment stints
max_stints = max(len(group) for group in employment_groups.values())

# Iterate through the rows and extract each employment stint for each person
for idx, row in filtered_df.iterrows():
    person_info = row[person_columns]

    for i in range(max_stints):
        employment_info = {}
        for base_col in employment_base_columns:
            if base_col in employment_groups and i < len(
                employment_groups[base_col]
            ):
                col = employment_groups[base_col][i]
                employment_info[base_col] = row[col]
            else:
                employment_info[base_col] = pd.NA

        # Check if at least one column in this set is not null
        if any(pd.notna(value) for value in employment_info.values()):
            # Combine person info and employment info
            combined_info = pd.concat([person_info, pd.Series(employment_info)])
            cleaned_df = pd.concat(
                [cleaned_df, combined_info.to_frame().T], ignore_index=True
            )


cleaned_df.reset_index(drop=True, inplace=True)

In [24]:
def rename_cols(df):
    df = df.rename(
        columns={
            "Person Certification #": "person_nbr",
            "Person First Name": "first_name",
            "Person Middle Name": "middle_name",
            "Person Last Name": "last_name",
            "Person Suffix": "suffix",
            "Person Gender": "sex",
            "Person EEOC Category": "race",
            "Year of Birth": "year_of_birth",
            "Employment Start Date": "start_date",
            "Employment End Date": "end_date",
            "Employment Employment Type*": "employment_type",
            "Employment Status": "employment_status",
            "Employing Organization Name": "agency_name",
            "Employing Organization Agency Type": "agency_type",
            "Employment Title/Rank (Current)": "rank",
        }
    )

    df = df.fillna("")

    df = df[
        [
            "person_nbr",
            "first_name",
            "middle_name",
            "last_name",
            "suffix",
            "sex",
            "year_of_birth",
            "race",
            "start_date",
            "end_date",
            "employment_type",
            "rank",
            "employment_status",
            "agency_name",
            "agency_type",
        ]
    ]
    return df


def clean_dates(df):
    df.loc[:, "start_date"] = df.start_date.astype(str).str.replace(
        r"(\w+):(\w+):(\w+)", "", regex=True
    )
    df.loc[:, "end_date"] = df.end_date.astype(str).str.replace(
        r"(\w+):(\w+):(\w+)", "", regex=True
    )
    return df


def clean_race(df):
    df.loc[:, "race"] = df.race.str.replace(r"Caucasian", "white", regex=False)
    return df


def clean_agency(df):
    df.loc[:, "agency_name"] = (
        df.agency_name.str.lower()
        .str.strip()
        .str.replace(r"dept\.", "department", regex=True)
        .str.replace(r"&", "and", regex=False)
        .str.replace(r"\bpd\b", "police department", regex=True)
        .str.replace(r"(\w+)  (\w+)", r"\1 \2", regex=True)
        .str.replace(r"mex\.", "mexico", regex=True)
    )
    df = df[~((df.agency_name.str.contains("fire")))]
    df.loc[:, "agency_name"] = df.agency_name.str.title()
    return df


def clean_special_characters(df):
    for column in df.columns:
        if df[column].dtype == "object":
            df[column] = df[column].apply(
                lambda x: "".join(
                    char for char in str(x) if ord(char) >= 32 or char == "\n"
                )
            )
    return df


def trim_whitespace(df):
    for column in df.columns:
        if df[column].dtype == "object":
            df[column] = df[column].str.strip()
    return df


def standardize_dates(df):
    date_columns = ["start_date", "end_date"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors="coerce").dt.strftime(
            "%Y-%m-%d"
        )
    return df


def handle_newlines(df):
    for column in df.columns:
        if df[column].dtype == "object":
            df[column] = df[column].str.replace("\n", " ")
    return df


def ensure_utf8(df):
    for column in df.columns:
        if df[column].dtype == "object":
            df[column] = df[column].apply(
                lambda x: (
                    x.encode("utf-8", errors="ignore").decode("utf-8")
                    if isinstance(x, str)
                    else x
                )
            )
    return df

In [25]:
df = (
    cleaned_df.pipe(rename_cols)
    .pipe(clean_dates)
    .pipe(clean_race)
    .pipe(clean_agency)
    .pipe(clean_special_characters)
    .pipe(trim_whitespace)
    .pipe(standardize_dates)
    .pipe(handle_newlines)
    .pipe(ensure_utf8)
)

df.sort_values("person_nbr").head(40)

/opt/homebrew/lib/python3.12/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


,person_nbr,first_name,middle_name,last_name,suffix,sex,year_of_birth,race,start_date,end_date,employment_type,rank,employment_status,agency_name,agency_type
4560,00-0001-P,Joshua,D,Anderson,,Male,1977,white,2000-05-22,2019-10-31,Full Time,Sergeant,Retired,Bernalillo Police Department,Law Enforcement Agency
4561,00-0001-P,Joshua,D,Anderson,,Male,1977,white,1999-07-26,2000-04-17,,Patrol Officer,Resigned,Farmington Police Department,Law Enforcement Agency
16416,00-0002-P,Phillip,B,Francisco,,Male,1976,Hispanic,2022-01-04,NaN,Full Time,Chief,Active,Bloomfield Police Department,Law Enforcement Agency
16415,00-0002-P,Phillip,B,Francisco,,Male,1976,Hispanic,2001-07-09,2014-08-25,,Deputy Sheriff,Resigned,San Juan County Sheriff'S Office,Law Enforcement Agency
16414,00-0002-P,Phillip,B,Francisco,,Male,1976,Hispanic,2014-08-25,2016-08-01,,[Missing Title/Rank],Resigned,Farmington Police Department,Law Enforcement Agency
16413,00-0002-P,Phillip,B,Francisco,,Male,1976,Hispanic,1999-07-18,2001-06-23,,Patrol Officer,Resigned,Aztec Police Department,Law Enforcement Agency
4689,00-0003-P,Angela,D,Gettler,,Female,1975,Native American,1999-07-26,2000-04-29,,Deputy Sheriff,Termination,San Juan County Sheriff'S Office,Law Enforcement Agency
4688,00-0003-P,Angela,D,Gettler,,Female,1975,Native American,2001-11-26,2003-05-15,,Patrol Officer,Termination,Bloomfield Police Department,Law Enforcement Agency
14213,00-0004-P,Robert,D,Gross,,Male,1970,white,1999-07-26,2016-08-31,,Deputy Sheriff,Retired,San Juan County Sheriff'S Office,Law Enforcement Agency
3595,00-0005-P,Bernie,J,Guffey,,Male,1970,white,1999-06-04,2001-10-31,,Patrol Officer,Termination,Aztec Police Department,Law Enforcement Agency


In [26]:
df.sort_values("person_nbr").tail(40)

,person_nbr,first_name,middle_name,last_name,suffix,sex,year_of_birth,race,start_date,end_date,employment_type,rank,employment_status,agency_name,agency_type
1037,99-0405-P,Todd,D,Cravens,,Male,1969,white,1999-07-19,NaN,Full Time,Patrol Officer,Active,Albuquerque Police Department,Law Enforcement Agency
2087,99-0406-P,Mark,R,Dietz,,Male,1972,white,1999-07-19,2014-06-13,,Patrol Officer,Retired,Albuquerque Police Department,Law Enforcement Agency
2549,99-0407-P,Simon,E,Drobik,,Male,1972,white,1999-07-19,2020-07-10,,Patrol Officer,Retired,Albuquerque Police Department,Law Enforcement Agency
2993,99-0408-P,Dawne,B,Roberto,,Female,1972,white,1999-07-19,2015-06-20,,Patrol Officer,Termination,Albuquerque Police Department,Law Enforcement Agency
2994,99-0408-P,Dawne,B,Roberto,,Female,1972,white,2013-09-16,2015-09-02,,Officer,Resigned,Bosque Farms Police Department,Law Enforcement Agency
1181,99-0409-P,Sean,P,Higdon,,Male,1972,white,1999-07-19,NaN,Full Time,Sergeant,Active,Albuquerque Police Department,Law Enforcement Agency
2830,99-0410-P,Heather,C,Chavez,,Female,1976,white,1999-07-19,2021-05-08,,Patrol Officer,Retired,Albuquerque Police Department,Law Enforcement Agency
2168,99-0411-P,David,C,Hinson,,Male,1969,white,1999-07-17,2022-08-13,,Patrol Officer,Retired,Albuquerque Police Department,Law Enforcement Agency
2136,99-0412-P,Lance,A,Hoisington,,Male,1976,white,1999-07-19,2017-06-16,,Patrol Officer,Retired,Albuquerque Police Department,Law Enforcement Agency
2171,99-0413-P,Shawn,P,Lockey,,Male,1977,white,1999-07-19,2019-09-28,,Sergeant,Retired,Albuquerque Police Department,Law Enforcement Agency


In [28]:
df.to_csv("../data/output/new-mexico-preprocessed.csv", index=False)